# MESA Batch Inlist Generator

This notebook creates batch inlists for MESA simulations from a CSV file of parameters.

## Instructions

1. Upload your CSV file with the parameters using the file upload widget below
2. Run all cells in sequence
3. Download the generated inlist files as a zip file
4. Extract the zip to your MESA work directory

In [ ]:
# Install required packages
!pip install -q ipywidgets

In [ ]:
# Import required libraries
import os
import csv
import re
import io
import zipfile
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML

## 1. Upload CSV File

Upload your CSV file containing the MESA model parameters. The CSV should have the following columns:
- Name
- Initial Mass [Msol]
- Initial Metallicity
- Overshoot Scheme
- Overshoot Parameter (f_ov)
- Overshoot f0

In [ ]:
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]  # Get the filename
print(f"Uploaded: {csv_filename}")

## 2. Provide MESA Template Inlist

Now please upload your template `inlist_project` file. This is the base file that will be modified for each parameter set.

In [ ]:
uploaded_template = files.upload()
template_filename = list(uploaded_template.keys())[0]  # Get the filename
print(f"Uploaded template: {template_filename}")

# Read the template content
with open(template_filename, 'r') as f:
    template_content = f.read()

## 3. Create Batch Inlists

This function will create an inlist file for each row in the CSV file.

In [ ]:
def create_batch_inlists(csv_file, template_content):
    """Create batch inlists from parameters in CSV file"""
    # Define paths
    batch_dir = "batch_inlists"

    # Create batch directory if it doesn't exist
    os.makedirs(batch_dir, exist_ok=True)

    # Dictionary to store inlist files
    inlist_files = {}

    # Read CSV file, skipping header
    with open(csv_file, 'r', newline='') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row

        for row in reader:
            # Unpack row or skip if too short
            if len(row) < 6:
                continue

            name = row[0].strip()
            mass = row[1].strip()
            metallicity = row[2].strip()
            scheme = row[3].strip()
            fov = row[4].strip()
            f0 = row[5].strip()

            # Skip rows with missing data
            if not mass or not metallicity:
                continue

            # Create descriptive filename encoding the parameters
            if scheme.lower() in ["no overshooting", "none", "no overshoot"]:
                outfile = os.path.join(batch_dir, f"inlist_M{mass}_Z{metallicity}_noovs.inp")
                outfile_basename = f"inlist_M{mass}_Z{metallicity}_noovs.inp"
                ovs_option = "none"
            else:
                outfile = os.path.join(batch_dir, f"inlist_M{mass}_Z{metallicity}_{scheme}_fov{fov}_f0{f0}.inp")
                outfile_basename = f"inlist_M{mass}_Z{metallicity}_{scheme}_fov{fov}_f0{f0}.inp"
                ovs_option = scheme

            print(f"Creating {outfile_basename}...")

            # Start with the template content
            content = template_content

            # Update parameters in the inlist file
            content = re.sub(r'initial_mass = [0-9]*(\.?[0-9]*)?', f'initial_mass = {mass}', content)
            content = re.sub(r'initial_z = [0-9]*(\.?[0-9]*)?', f'initial_z = {metallicity}', content)
            content = re.sub(r'Zbase = [0-9]*(\.?[0-9]*)?', f'Zbase = {metallicity}', content)

            # Handle save_model_filename
            model_filename = f"M{mass}_Z{metallicity}"
            if ovs_option != "none":
                model_filename = f"{model_filename}_{scheme}_fov{fov}_f0{f0}"
            else:
                model_filename = f"{model_filename}_noovs"

            content = re.sub(r"save_model_filename = '.*'", f"save_model_filename = '{model_filename}.mod'", content)

            # Handle overshoot parameters
            if ovs_option == "none":
                # Comment out all overshoot lines
                content = re.sub(r'^(\s*overshoot_scheme)', r'!\1', content, flags=re.MULTILINE)
                content = re.sub(r'^(\s*overshoot_zone_type)', r'!\1', content, flags=re.MULTILINE)
                content = re.sub(r'^(\s*overshoot_zone_loc)', r'!\1', content, flags=re.MULTILINE)
                content = re.sub(r'^(\s*overshoot_bdy_loc)', r'!\1', content, flags=re.MULTILINE)
                content = re.sub(r'^(\s*overshoot_f\()', r'!\1', content, flags=re.MULTILINE)
                content = re.sub(r'^(\s*overshoot_f0\()', r'!\1', content, flags=re.MULTILINE)
            else:
                # Update overshoot parameters
                content = re.sub(r'!*\s*overshoot_scheme\(1\) = .*', f'     overshoot_scheme(1) = \'{scheme}\'', content)
                content = re.sub(r'!*\s*overshoot_zone_type\(1\) = .*', f'     overshoot_zone_type(1) = \'any\'', content)
                content = re.sub(r'!*\s*overshoot_zone_loc\(1\) = .*', f'     overshoot_zone_loc(1) = \'core\'', content)
                content = re.sub(r'!*\s*overshoot_bdy_loc\(1\) = .*', f'     overshoot_bdy_loc(1) = \'top\'', content)
                content = re.sub(r'!*\s*overshoot_f\(1\) = .*', f'     overshoot_f(1) = {fov}', content)
                content = re.sub(r'!*\s*overshoot_f0\(1\) = .*', f'     overshoot_f0(1) = {f0}', content)

            # Store the content in our dictionary
            inlist_files[outfile_basename] = content

    print(f"Created {len(inlist_files)} batch inlist files.")
    return inlist_files

In [ ]:
# Run the batch inlist creation
inlist_files = create_batch_inlists(csv_filename, template_content)

# Display the first few lines of a sample inlist file (if any were created)
if inlist_files:
    sample_filename = list(inlist_files.keys())[0]
    sample_content = inlist_files[sample_filename]
    print(f"\nSample of generated inlist file '{sample_filename}':\n")
    print("\n".join(sample_content.split("\n")[:20]) + "\n...")
else:
    print("No inlist files were created. Check your CSV file.")

## 4. Create and Download Zip File

Pack all the inlist files into a zip for easy download.

In [ ]:
def create_zip_file(inlist_files):
    """Create a zip file containing all the inlist files"""
    zip_filename = "batch_inlists.zip"

    # Create zip file in memory
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        # Create a batch_inlists directory in the zip
        for filename, content in inlist_files.items():
            zip_file.writestr(os.path.join("batch_inlists", filename), content)

    # Save the zip file
    with open(zip_filename, 'wb') as f:
        f.write(zip_buffer.getvalue())

    return zip_filename

In [ ]:
# Create the zip file
zip_filename = create_zip_file(inlist_files)
print(f"Created zip file: {zip_filename}")

# Provide download link
files.download(zip_filename)

## 5. Installation Instructions

After downloading the zip file, follow these steps to use the inlist files with MESA:

1. Extract the `batch_inlists.zip` file
2. Copy the `batch_inlists` folder to the `batch_runs` directory in your MESA work directory
3. Use the `run_batch.py` script or the MESA Run Batch notebook to run simulations with these inlists

The directory structure should look like:
```
mesa_work_directory/
├── batch_runs/
│   ├── batch_inlists/
│   │   ├── inlist_M2_Z0.0140_noovs.inp
│   │   ├── inlist_M2_Z0.0140_exponential_fov0.01_f00.001.inp
│   │   └── ... (other inlist files)
│   └── runs/
├── inlist
├── inlist_project
├── inlist_pgstar
└── ... (other MESA files)
```